# 6.5. Small Business Owners in the United States🇺🇸

In this assignment, we will focus on business owners in the United States. We'll start by examining some demographic characteristics of the group, such as age, income category, and debt vs home value. Then we'll select high-variance features, and create a clustering model to divide small business owners into subgroups. Finally, we'll create some visualizations to highlight the differences between these subgroups.

## Import Libraries

In [ ]:
# Import libraries here

import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import wqet_grader
from IPython.display import VimeoVideo
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

## Prepare Data

### Import Data

In [ ]:
df = pd.read_csv("data/SCFP2019.csv.gz")
print("df shape:", df.shape)
df.head()

### Explore

In [ ]:
df["HBUS"].mean()

In [ ]:
pct_biz_owners = df["HBUS"].mean()
print("% of business owners in df:", pct_biz_owners)

In [ ]:
inccat_dict = {
    1: "0-20",
    2: "21-39.9",
    3: "40-59.9",
    4: "60-79.9",
    5: "80-89.9",
    6: "90-100",
}

df_inccat = (
    df["INCCAT"]
    .replace(inccat_dict)
    .groupby(df["HBUS"])
    .value_counts(normalize=True)
    .rename("frequency")
    .to_frame()
    .reset_index()
)

df_inccat

In [ ]:
# Create bar chart of `df_inccat`
sns.barplot(
    x="INCCAT",
    y="frequency",
    hue="HBUS",
    data=df_inccat,
    order=inccat_dict.values()
)
plt.xlabel("Income Category")
plt.ylabel("Frequency (%)")
plt.title("Income Distribution: Business Owners vs. Non-Business Owners");

In [ ]:
# Plot "HOUSES" vs "DEBT" with hue=label
sns.scatterplot(x= df["DEBT"]/1e6,
                y= df["HOUSES"]/1e6,
                palette="deep"
               )
plt.xlabel("Household Debt [$1M]")
plt.ylabel("Home Value [$1M]")
plt.title("Home Value vs. Household Debt");

In [ ]:
mask = (df["HBUS"] == 1) & (df["INCOME"]<500000)
df_small_biz = df[mask]
print("df_small_biz shape:", df_small_biz.shape)
df_small_biz.head()

In [ ]:
# Plot histogram of "AGE"
df_small_biz["AGE"].hist(bins=10)
plt.xlabel("Age"),
plt.ylabel("Frequency (count)"),
plt.title("Small Business Owners: Age Distribution");

In [ ]:
df_small_biz.var().sort_values().tail(10)

In [ ]:
# Calculate variance, get 10 largest features
top_ten_var = df_small_biz.var().sort_values().tail(10)
top_ten_var

In [ ]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_var,
    y= top_ten_var.index,
    title = "SCF: High Variance Features"
)

In [ ]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df,
    x="NHNFIN",
    title="Distribution of Non-home, Non-Financial Assets"
)
fig.update_layout(xaxis_title="Value [$]")

fig.show()

In [ ]:
# Calculate trimmed variance (to remove outliers)
top_ten_trim_var = df_small_biz.apply(trimmed_var, limits=(0.1, 0.1)).sort_values().tail(10)
top_ten_trim_var

In [ ]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_var,
    y= top_ten_trim_var.index,
    title = "SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")

fig.show()

In [ ]:
high_var_cols = top_ten_trim_var.tail(5).index.to_list()
high_var_cols

### Split

In [ ]:
X = df_small_biz[high_var_cols]
print("X shape:", X.shape)

## Build Model

### Iterate

In [ ]:
n_clusters = range(2, 13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    #Build model
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    #Train Model
    model.fit(X)
    #Calculate inertia
    inertia_errors.append(model.named_steps["kmeans"].inertia_)
    #Calculate silhouette score
    silhouette_scores.append(
        silhouette_score(X,model.named_steps["kmeans"].labels_)
    )

print("Inertia:", inertia_errors[:11])
print()
print("Silhouette Scores:", silhouette_scores[:3])

In [ ]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=inertia_errors, title="K-Means: Inertia vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters (k)", yaxis_title="Inertia")

fig.show()

In [ ]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=silhouette_scores, title="K-Means Model: Silhouette Score vs Number of Clusters"
)
fig.update_layout(
    xaxis_title="Number of Clusters (k)", yaxis_title="Silhouette Score"
)

fig.show()

In [ ]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=3, random_state=42)
)
# Fit model to data
final_model.fit(X)

## Communicate

In [ ]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title= "Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")
fig.show()

In [ ]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=["PC1", "PC2"])

print("X_pca shape:", X_pca.shape)
X_pca.head()

In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x="PC1",
    y="PC2",
    color=labels,
    title="PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()